In [ ]:
#1.Seurat.R
dir.create("./outdir")
dir.create("./outdir/1.QC")
dir.create("./outdir/2.Cluster")
dir.create("./outdir/3.Diff_Marker")
dir.create("./outdir/others")
library(Seurat)
library(dplyr)
library(sctransform)
library(ggplot2)
counts <- Read10X(data.dir = "./test_data/GSM5028741_Bone_marrow_4",gene.column=2)
seurat_obj<- CreateSeuratObject(counts, min.features = 200, project = "GSM5028741_Bone_marrow_4")
all_genes <- rownames(seurat_obj)
nonhs_genes <- grep("^NONHS", all_genes, value = TRUE)
non_nonhs_genes <- setdiff(all_genes, nonhs_genes)
lnc_data_GSM5028741_Bone_marrow_4<-subset(seurat_obj, features = nonhs_genes)
data_GSM5028741_Bone_marrow_4<-subset(seurat_obj, features = non_nonhs_genes)
saveRDS(lnc_data_GSM5028741_Bone_marrow_4,file="./outdir/lnc_data_GSM5028741_Bone_marrow_4.rds")
#Proportion of mitochondria
mito.features <- grep(pattern = "^MT-", x = rownames(x = data_GSM5028741_Bone_marrow_4), value = TRUE,ignore.case=TRUE)
percent.mito <- Matrix::colSums(x = GetAssayData(object = data_GSM5028741_Bone_marrow_4, slot = "counts")[mito.features,])/Matrix::colSums(x = GetAssayData(object = data_GSM5028741_Bone_marrow_4, slot = "counts"))
data_GSM5028741_Bone_marrow_4[["percent.mito"]] <- percent.mito  # The [[ operator can add columns to object metadata
#Proportion of ribosomes
ribosome.features1 <- grep(pattern = "^RPL", x = rownames(x = data_GSM5028741_Bone_marrow_4), value = TRUE,ignore.case=TRUE)
ribosome.features2 <- grep(pattern = "^RPS", x = rownames(x = data_GSM5028741_Bone_marrow_4), value = TRUE,ignore.case=TRUE)
ribosome.features<-c(ribosome.features1,ribosome.features2)
percent.ribosome <- Matrix::colSums(x = GetAssayData(object = data_GSM5028741_Bone_marrow_4, slot = "counts")[ribosome.features,])/Matrix::colSums(x = GetAssayData(object = data_GSM5028741_Bone_marrow_4, slot = "counts"))
data_GSM5028741_Bone_marrow_4[["percent.ribosome"]] <- percent.ribosome  # The [[ operator can add columns to object metadata
# Distribution of cell gene number, UMI number, and percentage of mitochondria
pdf(file="./outdir/1.QC/QC.pdf")
p<-VlnPlot(object = data_GSM5028741_Bone_marrow_4, pt.size = 0.1, features = c("nFeature_RNA", "nCount_RNA", "percent.mito", "percent.ribosome"),ncol = 2)
print(p)
p2<-FeatureScatter(object = data_GSM5028741_Bone_marrow_4, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
print(p2)
dev.off()
data_GSM5028741_Bone_marrow_4 <- subset(x = data_GSM5028741_Bone_marrow_4, subset = nFeature_RNA > 200 & nFeature_RNA < 10000 & percent.mito < 0.3 & percent.ribosome < 1  )
data_GSM5028741_Bone_marrow_4$stim <- "GSM5028741_Bone_marrow_4"
# Normalizing the data
data_GSM5028741_Bone_marrow_4 <- NormalizeData(object = data_GSM5028741_Bone_marrow_4, normalization.method = "LogNormalize")
# FindVariableFeatures
data_GSM5028741_Bone_marrow_4 <- FindVariableFeatures(object = data_GSM5028741_Bone_marrow_4, selection.method = "vst", nfeatures = 2000)  
# ScaleData
data_GSM5028741_Bone_marrow_4 <- ScaleData(object = data_GSM5028741_Bone_marrow_4, features = rownames(x = data_GSM5028741_Bone_marrow_4) )
# CellCycleScoring
s.genes <- cc.genes$s.genes
g2m.genes <- cc.genes$g2m.genes
data_GSM5028741_Bone_marrow_4 <- CellCycleScoring(data_GSM5028741_Bone_marrow_4, s.features = s.genes, g2m.features = g2m.genes, set.ident = TRUE)
data_GSM5028741_Bone_marrow_4$CC.Difference <- data_GSM5028741_Bone_marrow_4$S.Score - data_GSM5028741_Bone_marrow_4$G2M.Score
# PCA
data_GSM5028741_Bone_marrow_4 <- RunPCA(object = data_GSM5028741_Bone_marrow_4, pcs.compute = 30, features = VariableFeatures(object = data_GSM5028741_Bone_marrow_4))

pdf(file="./outdir/1.QC/visualize_PCA_DimPlot.pdf")
DimPlot(object = data_GSM5028741_Bone_marrow_4)
dev.off()
data_GSM5028741_Bone_marrow_4 <- ProjectDim(object = data_GSM5028741_Bone_marrow_4)
pdf(file="./outdir/others/visualize_PCA_DimHeatmap.pdf")
DimHeatmap(object = data_GSM5028741_Bone_marrow_4, dims = 1:5, cells = 500, balanced = TRUE)
dev.off()
pdf(file="./outdir/others/determine_PCA.pdf")
data_GSM5028741_Bone_marrow_4 <- JackStraw(object = data_GSM5028741_Bone_marrow_4, num.replicate = 100)
data_GSM5028741_Bone_marrow_4 <- ScoreJackStraw(object = data_GSM5028741_Bone_marrow_4, dims = 1:20)
JackStrawPlot(object = data_GSM5028741_Bone_marrow_4, dims = 1:20)
ElbowPlot(object = data_GSM5028741_Bone_marrow_4,ndims=50)
dev.off()

ndim<-30
# Cluster 
# graph-based clustering
data_GSM5028741_Bone_marrow_4 <- FindNeighbors(object = data_GSM5028741_Bone_marrow_4, dims = 1:ndim)
data_GSM5028741_Bone_marrow_4 <- FindClusters(object = data_GSM5028741_Bone_marrow_4, resolution = 1)
data_GSM5028741_Bone_marrow_4 <- RunUMAP(object = data_GSM5028741_Bone_marrow_4, dims = 1:ndim, check_duplicates = FALSE)
pdf(file="./outdir/2.Cluster/umap.pdf")
DimPlot(object = data_GSM5028741_Bone_marrow_4, reduction = "umap",label = TRUE)
dev.off()
data_GSM5028741_Bone_marrow_4 <- RunTSNE(object = data_GSM5028741_Bone_marrow_4, dims = 1:ndim, check_duplicates = FALSE)
pdf(file="./outdir/2.Cluster/tsne.pdf")
DimPlot(object = data_GSM5028741_Bone_marrow_4, reduction = "tsne",label = TRUE)
dev.off()
write.table(data_GSM5028741_Bone_marrow_4@active.ident,file="./outdir/2.Cluster/Cluster.xls",sep="\t",quote=F)
dir.create("./outdir/plot")
write.table(data_GSM5028741_Bone_marrow_4@reductions$umap@cell.embeddings,'./outdir/plot/umap.reductions.xls',quote=F,sep="\t")
write.table(data_GSM5028741_Bone_marrow_4@reductions$tsne@cell.embeddings,'./outdir/plot/tsne.reductions.xls',quote=F,sep="\t")
write.table(data_GSM5028741_Bone_marrow_4@meta.data,file="meta.data",quote=FALSE, sep="\t")
avg<-AverageExpression(data_GSM5028741_Bone_marrow_4,use.scale = FALSE)
write.table(avg$RNA,file="./outdir/cluster_avgExp_normalized.xls",sep="\t",quote=F)
saveRDS(data_GSM5028741_Bone_marrow_4,file="./outdir/data_GSM5028741_Bone_marrow_4.rds")
#Findmarker
data_GSM5028741_Bone_marrow_4.markers <- SeuratWrappers::RunPrestoAll(object = data_GSM5028741_Bone_marrow_4, only.pos = TRUE, min.pct = 0.25, thresh.use = 0.25, test.use="wilcox")
data_GSM5028741_Bone_marrow_4.markers2 <- SeuratWrappers::RunPrestoAll(object = data_GSM5028741_Bone_marrow_4, min.pct = 0.25, thresh.use = 0.25, test.use="wilcox")
write.table(data_GSM5028741_Bone_marrow_4.markers,file="./outdir/3.Diff_Marker/All_markers_only_pos.xls",quote=F,sep="\t")
write.table(data_GSM5028741_Bone_marrow_4.markers2,file="./outdir/3.Diff_Marker/All_markers.xls",quote=F,sep="\t")
# cluster TOP10 Marker
pdf(file="./outdir/3.Diff_Marker/marker_heatmap_top10.pdf",width=8,height=15)
top10 <- data_GSM5028741_Bone_marrow_4.markers %>% group_by(cluster) %>% top_n(n = 10, wt = avg_log2FC)
DoHeatmap(object = data_GSM5028741_Bone_marrow_4, features = top10$gene, size=1.5) + NoLegend()
dev.off()
pdf(file="./outdir/3.Diff_Marker/marker_heatmap_top5.pdf",width=8,height=12)
top5 <- data_GSM5028741_Bone_marrow_4.markers %>% group_by(cluster) %>% top_n(n = 5, wt = avg_log2FC)
DoHeatmap(object = data_GSM5028741_Bone_marrow_4, features = top5$gene, size=1.5) + NoLegend()
dev.off()

#2.sctype.R
library(knitr)
library(dplyr)
library(Seurat)
library('HGNChelper')
source("./sc-type-master/R/gene_sets_prepare.R")
source("./sc-type-master/R/sctype_score_.R")
data_GSM5028741_Bone_marrow_4<-readRDS("./outdir/data_GSM5028741_Bone_marrow_4.rds")
data_GSM5028741_Bone_marrow_4 <- ScaleData(object = data_GSM5028741_Bone_marrow_4)
scRNAseqData <- data_GSM5028741_Bone_marrow_4@assays$RNA@scale.data
gs_list = gene_sets_prepare("./sc-type-master/ScTypeDB_short.xlsx", "Human_All") # e.g. Immune system, Liver, Pancreas, Kidney, Eye, Brain
# get cell-type by cell matrix
es.max = sctype_score(scRNAseqData = scRNAseqData, scaled = TRUE, gs = gs_list$gs_positive,gs2 = gs_list$gs_negative)
# merge by cluster
cL_resutls = do.call("rbind", lapply(unique(data_GSM5028741_Bone_marrow_4@meta.data$seurat_clusters), function(cl) {
    es.max.cl = sort(rowSums(es.max[, rownames(data_GSM5028741_Bone_marrow_4@meta.data[data_GSM5028741_Bone_marrow_4@meta.data$seurat_clusters ==
        cl, ])]), decreasing = !0)
    head(data.frame(cluster = cl, type = names(es.max.cl), scores = es.max.cl, ncells = sum(data_GSM5028741_Bone_marrow_4@meta.data$seurat_clusters ==
        cl)), 10)
}))
sctype_scores = cL_resutls %>% group_by(cluster) %>% top_n(n = 1, wt = scores)
# set low-confident (low ScType score) clusters to 'unknown'
sctype_scores$type[as.numeric(as.character(sctype_scores$scores)) < sctype_scores$ncells/30] = "Unknown"
print(sctype_scores[, 1:3])
write.table(sctype_scores[,1:3],file="./outdir/cluster_cell_type.xls",sep="\t",quote=F,col.names=NA)
#UMAP
data_GSM5028741_Bone_marrow_4@meta.data$celltype = ""
for (j in unique(sctype_scores$cluster)) {
    cl_type = sctype_scores[sctype_scores$cluster == j, ]
    data_GSM5028741_Bone_marrow_4@meta.data$celltype[data_GSM5028741_Bone_marrow_4@meta.data$seurat_clusters == j] = as.character(cl_type$type[1])
}
pdf("./outdir/sctype_umap.pdf",width=10)
DimPlot(data_GSM5028741_Bone_marrow_4, reduction = "umap", label = TRUE, repel = TRUE, group.by = "celltype")
dev.off()
pdf("./outdir/sctype_tsne.pdf",width=10)
DimPlot(data_GSM5028741_Bone_marrow_4, reduction = "tsne", label = TRUE, repel = TRUE, group.by = "celltype")
dev.off()
write.table(data_GSM5028741_Bone_marrow_4@meta.data,file="./outdir/cell_type.xls",sep="\t",quote=F)

#3.singleR.R
library(SingleR)
library(SingleCellExperiment)
library(scater)
library(Seurat)
data_GSM5028741_Bone_marrow_4<-readRDS("./outdir/data_GSM5028741_Bone_marrow_4.rds")
counts <- data_GSM5028741_Bone_marrow_4@assays$RNA@counts
#e<-as.matrix(counts)
e<-counts
mytest<-SingleCellExperiment(assays = list(counts = e))
myref<-readRDS('./HumanPrimaryCellAtlasData.rds')
common <- intersect(rownames(mytest), rownames(myref))
myref<-myref[common,]
mytest<-mytest[common,]
mytest<-logNormCounts(mytest)
pred.main <- SingleR(test = mytest, ref = myref, labels =myref$label.main ,assay.type.ref = "logcounts")
#pdf(file="GSM5028741_Bone_marrow_4_cell_type_main.pdf")
#plotScoreHeatmap(pred.main,clusters=data_GSM5028741_Bone_marrow_4@active.ident,order.by.clusters = TRUE, show.labels=F)
#dev.off()
write.table(pred.main@rownames,file="./outdir/rowname",sep="	",quote=F)
write.table(pred.main$labels,file="./outdir/labels",sep="	",quote=F)
pred.fine <- SingleR(test = mytest, ref = myref, labels =myref$label.fine ,assay.type.ref = "logcounts")
#pdf(file="GSM5028741_Bone_marrow_4_cell_type_fine.pdf")
#plotScoreHeatmap(pred.fine,clusters=data_GSM5028741_Bone_marrow_4@active.ident,order.by.clusters = TRUE, show.labels=F)
#dev.off()
#save(pred.main,file='pred.main.Rdata')
#save(pred.fine,file='pred.fine.Rdata')

#write.table(singler$other,file="GSM5028741_Bone_marrow_4_cell_type2.xls",sep="\t",quote=F)

#4.singleR_deal.sh
/usr/bin/perl ./cluster2type.pl ./outdir/rowname ./outdir/labels ./outdir >./outdir/CellType.xls
#5.merge_singleRsctype.pl
if(-f "./outdir/CellType.xls"){
        open IN,"./outdir/CellType.xls";<IN>;
        while(<IN>){
                chomp;
                my @a=split /\t/,$_;
                $cluster2type{$a[0]}=$a[1];
        }
        close IN;
        open IN,"./outdir/cell_type.xls";
        open OUT,">./outdir/cell_type_new.xls";
        my $head=<IN>;print OUT "$head";
        while(<IN>){
                chomp;
                my @a=split /\t/,$_;
                if($a[-1] eq 'Unknown'){
                        $a[-1]=$cluster2type{$a[-4]};
                }
                my $o=join("\t",@a);
                print OUT "$o\n";
        }
        close OUT;close IN;
}

#6.Seurat_final.R
library(knitr)
library(dplyr)
library(Seurat)
library(ggplot2)
dir.create("./outdir/result")
dir.create("./outdir/result/mRNA")
dir.create("./outdir/result/lncRNA")

data_GSM5028741_Bone_marrow_4<-readRDS("./outdir/data_GSM5028741_Bone_marrow_4.rds")
data_GSM5028741_Bone_marrow_4 <- ScaleData(object = data_GSM5028741_Bone_marrow_4)
lnc_data_GSM5028741_Bone_marrow_4<-readRDS("./outdir/lnc_data_GSM5028741_Bone_marrow_4.rds")

ct<-read.table("./outdir/cell_type_new.xls",head=T,sep="\t")
ct<-ct[rownames(data_GSM5028741_Bone_marrow_4@meta.data),]
data_GSM5028741_Bone_marrow_4@meta.data$celltype = ct$celltype
new_idents<-data_GSM5028741_Bone_marrow_4@meta.data$celltype
names(new_idents)<-row.names(data_GSM5028741_Bone_marrow_4@meta.data)
new_idents<-as.factor(new_idents)
Idents(data_GSM5028741_Bone_marrow_4) <- new_idents
data_GSM5028741_Bone_marrow_4.markers <- SeuratWrappers::RunPrestoAll(object = data_GSM5028741_Bone_marrow_4, only.pos = TRUE, min.pct = 0.25, thresh.use = 0.25, test.use="wilcox")
write.table(data_GSM5028741_Bone_marrow_4.markers,file="./outdir/result/mRNA/All_markers.xls",quote=F,sep="\t",col.names=NA)
pdf(file="./outdir/result/mRNA/marker_heatmap_top10.pdf",width=8,height=16)
top10 <- data_GSM5028741_Bone_marrow_4.markers %>% group_by(cluster) %>% top_n(n = 10, wt = avg_log2FC)
data_GSM5028741_Bone_marrow_4 <- ScaleData(object = data_GSM5028741_Bone_marrow_4, features = rownames(x = data_GSM5028741_Bone_marrow_4))
DoHeatmap(object = data_GSM5028741_Bone_marrow_4, features = top10$gene, size=1.5) + NoLegend() + scale_fill_gradientn(colors = c("blue", "white", "red"))
dev.off()
avg<-AverageExpression(data_GSM5028741_Bone_marrow_4,assays='RNA')
write.table(avg$RNA,file="./outdir/result/mRNA/celltype_avgExp.xls",sep="\t",quote=F,col.names=NA)
data_GSM5028741_Bone_marrow_4@assays$RNA@scale.data<-matrix(,ncol=0,nrow=0)
pdf("./outdir/result/mRNA/umap.pdf",width=10)
DimPlot(data_GSM5028741_Bone_marrow_4, reduction = "umap", label = TRUE, repel = TRUE, group.by = "celltype")
dev.off()
pdf("./outdir/result/mRNA/tsne.pdf",width=10)
DimPlot(data_GSM5028741_Bone_marrow_4, reduction = "tsne", label = TRUE, repel = TRUE, group.by = "celltype")
dev.off()
saveRDS(data_GSM5028741_Bone_marrow_4,file="./outdir/mRNA.rds")
lnc_data_GSM5028741_Bone_marrow_4<-subset(lnc_data_GSM5028741_Bone_marrow_4,cells=row.names(data_GSM5028741_Bone_marrow_4@meta.data))
lnc_data_GSM5028741_Bone_marrow_4<- NormalizeData(object = lnc_data_GSM5028741_Bone_marrow_4, normalization.method = "LogNormalize")
lnc_data_GSM5028741_Bone_marrow_4 <- FindVariableFeatures(object = lnc_data_GSM5028741_Bone_marrow_4, selection.method = "vst", nfeatures = 2000)
lnc_data_GSM5028741_Bone_marrow_4 <- ScaleData(object = lnc_data_GSM5028741_Bone_marrow_4, features = rownames(x = lnc_data_GSM5028741_Bone_marrow_4))
lnc_data_GSM5028741_Bone_marrow_4 <- RunPCA(object = lnc_data_GSM5028741_Bone_marrow_4, pcs.compute = 30, features = VariableFeatures(object = lnc_data_GSM5028741_Bone_marrow_4))
ndim<-30
lnc_data_GSM5028741_Bone_marrow_4 <- FindNeighbors(object = lnc_data_GSM5028741_Bone_marrow_4, dims = 1:ndim)
lnc_data_GSM5028741_Bone_marrow_4 <- FindClusters(object = lnc_data_GSM5028741_Bone_marrow_4, resolution = 0.5)
lnc_data_GSM5028741_Bone_marrow_4 <- RunUMAP(object = lnc_data_GSM5028741_Bone_marrow_4, dims = 1:ndim, check_duplicates = FALSE)
lnc_data_GSM5028741_Bone_marrow_4 <- RunTSNE(object = lnc_data_GSM5028741_Bone_marrow_4, dims = 1:ndim, check_duplicates = FALSE) 
ct<-ct[rownames(lnc_data_GSM5028741_Bone_marrow_4@meta.data),]
lnc_data_GSM5028741_Bone_marrow_4@meta.data$celltype = ct$celltype
new_idents<-data_GSM5028741_Bone_marrow_4@meta.data$celltype
names(new_idents)<-row.names(lnc_data_GSM5028741_Bone_marrow_4@meta.data)
new_idents<-as.factor(new_idents)
Idents(lnc_data_GSM5028741_Bone_marrow_4) <- new_idents
lnc_data_GSM5028741_Bone_marrow_4.markers <- SeuratWrappers::RunPrestoAll(object = lnc_data_GSM5028741_Bone_marrow_4, only.pos = TRUE, min.pct = 0.25, thresh.use = 0.25, test.use="wilcox")
write.table(lnc_data_GSM5028741_Bone_marrow_4.markers,file="./outdir/result/lncRNA/All_markers.xls",quote=F,sep="\t",col.names=NA)
pdf(file="./outdir/result/lncRNA/marker_heatmap_top10.pdf",width=8,height=15)
top10 <- lnc_data_GSM5028741_Bone_marrow_4.markers %>% group_by(cluster) %>% top_n(n = 10, wt = avg_log2FC)
DoHeatmap(object = lnc_data_GSM5028741_Bone_marrow_4, features = top10$gene, size=1.5) + NoLegend() + scale_fill_gradientn(colors = c("blue", "white", "red"))
dev.off()
avg<-AverageExpression(lnc_data_GSM5028741_Bone_marrow_4,assays='RNA')
write.table(avg$RNA,file="./outdir/result/lncRNA/celltype_avgExp.xls",sep="	",quote=F,col.names=NA)
lnc_data_GSM5028741_Bone_marrow_4@assays$RNA@scale.data<-matrix(,ncol=0,nrow=0)
saveRDS(lnc_data_GSM5028741_Bone_marrow_4,file="./outdir/lncRNA.rds")
pdf("./outdir/result/lncRNA/umap.pdf",width=10)
DimPlot(lnc_data_GSM5028741_Bone_marrow_4, reduction = "umap", label = TRUE, repel = TRUE, group.by = "celltype")
dev.off()
pdf("./outdir/result/lncRNA/tsne.pdf",width=10)
DimPlot(lnc_data_GSM5028741_Bone_marrow_4, reduction = "tsne", label = TRUE, repel = TRUE, group.by = "celltype")
dev.off()

